In [6]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse
def parse_transcription(wav_file):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("addy88/wav2vec2-english-stt")
    model = Wav2Vec2ForCTC.from_pretrained("addy88/wav2vec2-english-stt")
    # load audio
    audio_input, sample_rate = sf.read(wav_file)
    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    print(transcription)

parse_transcription(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav') 


Some weights of the model checkpoint at addy88/wav2vec2-english-stt were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at addy88/wav2vec2-english-stt and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model 

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\dauma\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\internal\__init__.py)

In [4]:
import torch
import torchaudio
from torchaudio.transforms import Resample
import numpy as np
import torch.nn as nn
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

class Seq2SeqLossAccuracy:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        self.mse_loss = nn.MSELoss()

    def load_audio(self, filepath):
        waveform, sr = torchaudio.load(filepath)
        if sr != self.sample_rate:
            waveform = Resample(orig_freq=sr, new_freq=self.sample_rate)(waveform)
        return waveform.squeeze()

    def compute_mse_loss(self, pred_waveform, target_waveform):
        min_length = min(pred_waveform.size(0), target_waveform.size(0))
        pred_waveform = pred_waveform[:min_length]
        target_waveform = target_waveform[:min_length]
        return self.mse_loss(pred_waveform, target_waveform)

    def compute_dtw_distance(self, pred_waveform, target_waveform):
        pred_waveform = pred_waveform.numpy()
        target_waveform = target_waveform.numpy()
        distance, path = fastdtw(pred_waveform, target_waveform, dist=euclidean)
        return distance

    def evaluate(self, pred_filepath, target_filepath):
        pred_waveform = self.load_audio(pred_filepath)
        target_waveform = self.load_audio(target_filepath)

        mse_loss = self.compute_mse_loss(pred_waveform, target_waveform)
        dtw_distance = self.compute_dtw_distance(pred_waveform, target_waveform)

        return mse_loss.item(), dtw_distance

# Example usage
evaluator = Seq2SeqLossAccuracy(sample_rate=16000)

pred_filepath = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac.wav'
target_filepath = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\data\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'

mse_loss, dtw_distance = evaluator.evaluate(pred_filepath, target_filepath)
print(f"MSE Loss: {mse_loss:.4f}")
print(f"DTW Distance: {dtw_distance:.4f}")


ValueError: Input vector should be 1-D.